<a href="https://colab.research.google.com/github/kelvinmw1/Telco_customer_churn/blob/main/customer_churn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#First import the dataset

import pandas as pd
df = pd.read_csv('/content/my_dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')

#show a quick preview of the dataset
df.head()
#df.info()
#df.describe()

#checked for null values
df.isnull().sum()

#checked for duplicates
df.duplicated().sum()

#changed the data type of the total_charges column from object to numerical value

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')




In [51]:
#let us encode the categorical variables that the algorithm will use

#df = pd.get_dummies(df, columns=['gender','PhoneService','TechSupport'])
df.head()

#let us now scale the numerical features to improve the accuracy of our model
#first let us import the tools required

from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
num_columns = ['tenure','MonthlyCharges','TotalCharges']
df[num_columns] = scaler.fit_transform(df[num_columns])

df.head()



,customerID,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,TotalCharges,Churn,AvgMonthlyCharge,gender_Female,gender_Male,PhoneService_No,PhoneService_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes
0,7590-VHVEG,0,Yes,No,-1.277445,No phone service,DSL,No,Yes,No,...,-0.994194,No,0.778268,True,False,True,False,True,False,False
1,5575-GNVDE,0,No,No,0.066327,No,DSL,Yes,No,Yes,...,-0.173740,No,-2.619427,False,True,False,True,True,False,False
2,3668-QPYBK,0,No,No,-1.236724,No,DSL,Yes,Yes,No,...,-0.959649,Yes,0.775960,False,True,False,True,True,False,False
3,7795-CFOCW,0,No,No,0.514251,No phone service,DSL,Yes,No,Yes,...,-0.195248,No,-0.379674,False,True,True,False,False,False,True
4,9237-HQITU,0,No,No,-1.236724,No,Fiber optic,No,No,No,...,-0.940457,Yes,0.760442,True,False,False,True,True,False,False


In [59]:
#added a few features to add accuracy to the model
import numpy as np

df['AvgMonthlyCharge'] = df['TotalCharges'] / df['tenure'].replace(0, np.nan)

df.head()






,customerID,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,Churn,AvgMonthlyCharge,gender_Female,gender_Male,PhoneService_No,PhoneService_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,InteractionFrequency
0,7590-VHVEG,0,Yes,No,-1.277445,No phone service,DSL,No,Yes,No,...,No,0.778268,True,False,True,False,True,False,False,1
1,5575-GNVDE,0,No,No,0.066327,No,DSL,Yes,No,Yes,...,No,-2.619427,False,True,False,True,True,False,False,1
2,3668-QPYBK,0,No,No,-1.236724,No,DSL,Yes,Yes,No,...,Yes,0.775960,False,True,False,True,True,False,False,1
3,7795-CFOCW,0,No,No,0.514251,No phone service,DSL,Yes,No,Yes,...,No,-0.379674,False,True,True,False,False,False,True,1
4,9237-HQITU,0,No,No,-1.236724,No,Fiber optic,No,No,No,...,Yes,0.760442,True,False,False,True,True,False,False,1
